In [ ]:
import email
import os
from collections import Counter

from dotenv import load_dotenv
from imapclient import IMAPClient
from tqdm.autonotebook import tqdm

In [2]:
# Load environment variables from .env file
load_dotenv()

# iCloud IMAP server details
imap_host = 'imap.mail.me.com'
imap_port = 993

# User login details from environment variables
icloud_email = os.getenv('ICLOUD_USERNAME')
app_specific_password = os.getenv('ICLOUD_PASSWORD')

In [ ]:
# Connect to iCloud IMAP server
imap_server = IMAPClient(imap_host, use_uid=True)

# Login to the server
imap_server.login(icloud_email, app_specific_password)

In [ ]:
# Select the mailbox you want to search
imap_server.select_folder('INBOX')

# Search for all messages
messages = imap_server.search(['ALL'])

from_addresses = []

for msg_id in tqdm(messages[:20]):
    raw_message_data = imap_server.fetch(msg_id, ['BODY[HEADER]'])
    if raw_message_data and msg_id in raw_message_data and b'BODY[HEADER]' in raw_message_data[msg_id]:
        raw_headers = raw_message_data[msg_id][b'BODY[HEADER]']
        email_message = email.message_from_bytes(raw_headers)
        
        if email_message['from']:
            from_address = email.utils.parseaddr(email_message['from'])[1]
            from_addresses.append(from_address)


In [ ]:
# Count occurrences of each email address
email_counts = Counter(from_addresses)

# Print results
for email_address, count in email_counts.most_common():
    print(f"{email_address}: {count}")

imap_server.logout()
